In [35]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [36]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [37]:
tf.keras.backend.clear_session()

In [38]:
subject = 'Rio Grande Do Norte - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [39]:
data = pd.read_csv('2003_mo_model_input_RN.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Rio Grande do Norte - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio Grande Do Norte - Produção de Cimento (t),Rio Grande do Norte - PIB - Estadual,Rio Grande do Norte - PIB - Construção Civil,Rio Grande do Norte - PIB - Per Capita,Rio Grande do Norte - PIB - Preços de Mercado,Rio Grande do Norte - Desemprego,Rio Grande Do Norte - Consumo de Cimento (t)
0,2003-1,0.689488,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,27.698760,3.164847e+07,1.998664e+06,9.097985,2.879532e+07,8.294170,36.825
1,2003-2,0.689892,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,27.407645,3.168897e+07,2.001107e+06,9.101271,2.881033e+07,8.288224,34.088
2,2003-3,0.690297,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,27.377422,3.172947e+07,2.003550e+06,9.104557,2.882535e+07,8.282278,30.150
3,2003-4,0.690702,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,27.542280,3.176997e+07,2.005994e+06,9.107843,2.884036e+07,8.276332,31.224
4,2003-5,0.691107,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,27.944900,3.181046e+07,2.008437e+06,9.111128,2.885538e+07,8.270387,33.606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.557326,NaN,NaN,NaN,NaN,NaN,83.890
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94.910014,NaN,NaN,NaN,NaN,NaN,83.438
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94.800783,NaN,NaN,NaN,NaN,NaN,80.005
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94.752983,NaN,NaN,NaN,NaN,NaN,75.441


In [60]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Rio Grande do Norte - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio Grande Do Norte - Produção de Cimento (t),Rio Grande do Norte - PIB - Estadual,Rio Grande do Norte - PIB - Construção Civil,Rio Grande do Norte - PIB - Per Capita,Rio Grande do Norte - PIB - Preços de Mercado,Rio Grande do Norte - Desemprego
0,-2.184058,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.183161,-1.731345,-0.778207,-2.784513,-2.193066,-0.876481
1,-2.147573,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.192364,-1.711183,-0.728728,-2.703361,-2.146603,-0.878896
2,-2.111089,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.193319,-1.691021,-0.679248,-2.622210,-2.100140,-0.881311
3,-2.074605,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.188108,-1.670859,-0.629768,-2.541059,-2.053677,-0.883727
4,-2.038120,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.175380,-1.650697,-0.580289,-2.459908,-2.007214,-0.886142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.321156,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,0.389536,1.102751,-1.573216,0.457957,0.775635,1.052347
188,1.314269,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,0.398528,1.092237,-1.561825,0.427807,0.759230,1.051753
189,1.307381,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,0.414978,1.081724,-1.550435,0.397658,0.742826,1.051160
190,1.300494,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,0.429834,1.071210,-1.539044,0.367509,0.726421,1.050566


In [41]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      32.845
1      26.809
2      34.465
3      30.563
4      32.574
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Rio Grande Do Norte - Consumo de Cimento (t), Length: 240, dtype: float64

In [61]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Rio Grande do Norte - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio Grande Do Norte - Produção de Cimento (t),Rio Grande do Norte - PIB - Estadual,Rio Grande do Norte - PIB - Construção Civil,Rio Grande do Norte - PIB - Per Capita,Rio Grande do Norte - PIB - Preços de Mercado,Rio Grande do Norte - Desemprego
0,-2.184058,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.183161,-1.731345,-0.778207,-2.784513,-2.193066,-0.876481
1,-2.147573,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.192364,-1.711183,-0.728728,-2.703361,-2.146603,-0.878896
2,-2.111089,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.193319,-1.691021,-0.679248,-2.622210,-2.100140,-0.881311
3,-2.074605,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.188108,-1.670859,-0.629768,-2.541059,-2.053677,-0.883727
4,-2.038120,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.175380,-1.650697,-0.580289,-2.459908,-2.007214,-0.886142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.384282,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.046207,1.183768,-1.442416,0.625176,0.985411,1.213189
158,1.382302,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.000373,1.187707,-1.461501,0.642268,0.990347,1.201976
159,1.380323,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,0.957199,1.191647,-1.480587,0.659360,0.995283,1.190764
160,1.378343,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,0.908410,1.195587,-1.499673,0.676453,1.000219,1.179551


In [43]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      32.845
1      26.809
2      34.465
3      30.563
4      32.574
        ...  
157    52.722
158    63.109
159    49.356
160    55.725
161    50.796
Name: Rio Grande Do Norte - Consumo de Cimento (t), Length: 162, dtype: float64

In [44]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [45]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 14)

In [46]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Rio Grande do Norte - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio Grande Do Norte - Produção de Cimento (t),Rio Grande do Norte - PIB - Estadual,Rio Grande do Norte - PIB - Construção Civil,Rio Grande do Norte - PIB - Per Capita,Rio Grande do Norte - PIB - Preços de Mercado,Rio Grande do Norte - Desemprego
126,0.458594,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,1.480898,0.826780,0.124108,0.924042,0.982707,1.063612
127,0.463887,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,1.485581,0.843350,0.061862,0.903284,0.982590,1.075867
128,0.469179,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,1.491421,0.859920,-0.000383,0.882526,0.982474,1.088122
129,0.474472,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,1.490890,0.876489,-0.062629,0.861768,0.982357,1.100376
130,0.479764,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,1.486995,0.893059,-0.124874,0.841010,0.982241,1.112631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.321156,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,0.389536,1.102751,-1.573216,0.457957,0.775635,1.052347
188,1.314269,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,0.398528,1.092237,-1.561825,0.427807,0.759230,1.051753
189,1.307381,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,0.414978,1.081724,-1.550435,0.397658,0.742826,1.051160
190,1.300494,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,0.429834,1.071210,-1.539044,0.367509,0.726421,1.050566


In [47]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 14)

In [48]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [49]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(36, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(180,activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [50]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [51]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[3265960295, 2779794298, 1447449867, 2373431968, 4198794112, 1407920287, 59777146, 979624014, 1303862952, 1243708329, 2607592253, 498525153, 955386007, 913594416, 2991275930, 2624775097, 2652417522, 269272911, 1750046104, 1527118672, 786989890, 2227253753, 3213587546, 1404731207, 3924004834, 3390730562, 4181634592, 1469838545, 3753633302, 2189669479, 1616640909, 2942601094, 3699442179, 1182818249, 1227330420, 2389871637, 4225610938, 765859545, 737938872, 4056128666, 3985328767, 1043608272, 51409954, 493403958, 3569781195, 711496442, 2127529808, 2802197345, 1779759966, 9278365]


Step: 0 ___________________________________________
val_loss: 19.98521614074707
winner_seed: 3265960295


Step: 1 ___________________________________________
val_loss: 16.21103286743164
winner_seed: 2779794298


Step: 2 ___________________________________________
val_loss: 16.03199577331543
winner_seed: 1447449867


Step: 3 ___________________________________________
val_loss: 21.063753128051758


Step: 4 _____

In [52]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 136ms/step - loss: 3670.8098 - val_loss: 147.5316
Epoch 2/10000
4/4 [==============================] - 0s 13ms/step - loss: 178.9148 - val_loss: 304.1211
Epoch 3/10000
4/4 [==============================] - 0s 14ms/step - loss: 326.3365 - val_loss: 338.3629
Epoch 4/10000
4/4 [==============================] - 0s 13ms/step - loss: 191.5250 - val_loss: 121.9764
Epoch 5/10000
4/4 [==============================] - 0s 12ms/step - loss: 147.2624 - val_loss: 144.6637
Epoch 6/10000
4/4 [==============================] - 0s 12ms/step - loss: 129.1423 - val_loss: 97.3887
Epoch 7/10000
4/4 [==============================] - 0s 13ms/step - loss: 100.0368 - val_loss: 124.9658
Epoch 8/10000
4/4 [==============================] - 0s 12ms/step - loss: 259.4298 - val_loss: 127.9091
Epoch 9/10000
4/4 [==============================] - 0s 12ms/step - loss: 108.9071 - val_loss: 122.8112
Epoch 10/10000
4/4 [==============================] - 0s 11ms/s

4/4 [==============================] - 0s 12ms/step - loss: 22.4857 - val_loss: 47.8692
Epoch 160/10000
4/4 [==============================] - 0s 12ms/step - loss: 25.8487 - val_loss: 38.2109
Epoch 161/10000
4/4 [==============================] - 0s 14ms/step - loss: 24.1376 - val_loss: 30.9716
Epoch 162/10000
4/4 [==============================] - 0s 11ms/step - loss: 23.2685 - val_loss: 38.4320
Epoch 163/10000
4/4 [==============================] - 0s 14ms/step - loss: 23.2858 - val_loss: 32.5328
Epoch 164/10000
4/4 [==============================] - 0s 13ms/step - loss: 21.6485 - val_loss: 37.9536
Epoch 165/10000
4/4 [==============================] - 0s 14ms/step - loss: 24.9004 - val_loss: 36.5316
Epoch 166/10000
4/4 [==============================] - 0s 13ms/step - loss: 29.9117 - val_loss: 40.5452
Epoch 167/10000
4/4 [==============================] - 0s 14ms/step - loss: 32.5802 - val_loss: 35.5666
Epoch 168/10000
4/4 [==============================] - 0s 14ms/step - loss: 25.9

Epoch 238/10000
4/4 [==============================] - 0s 13ms/step - loss: 25.2650 - val_loss: 35.7692
Epoch 239/10000
4/4 [==============================] - 0s 14ms/step - loss: 22.8741 - val_loss: 32.0673
Epoch 240/10000
4/4 [==============================] - 0s 13ms/step - loss: 24.4218 - val_loss: 27.6851
Epoch 241/10000
4/4 [==============================] - 0s 14ms/step - loss: 23.3825 - val_loss: 29.9397
Epoch 242/10000
4/4 [==============================] - 0s 13ms/step - loss: 22.2348 - val_loss: 29.6007
Epoch 243/10000
4/4 [==============================] - 0s 14ms/step - loss: 20.0011 - val_loss: 35.9690
Epoch 244/10000
4/4 [==============================] - 0s 13ms/step - loss: 22.0390 - val_loss: 45.9040
Epoch 245/10000
4/4 [==============================] - 0s 12ms/step - loss: 24.9204 - val_loss: 31.1642
Epoch 246/10000
4/4 [==============================] - 0s 14ms/step - loss: 22.4502 - val_loss: 40.8443
Epoch 247/10000
4/4 [==============================] - 0s 14ms/s

Epoch 317/10000
4/4 [==============================] - 0s 14ms/step - loss: 19.8145 - val_loss: 32.3642
Epoch 318/10000
4/4 [==============================] - 0s 14ms/step - loss: 22.1026 - val_loss: 35.9476
Epoch 319/10000
4/4 [==============================] - 0s 14ms/step - loss: 21.0199 - val_loss: 36.1063
Epoch 320/10000
4/4 [==============================] - 0s 14ms/step - loss: 20.6812 - val_loss: 31.5772
Epoch 321/10000
4/4 [==============================] - 0s 13ms/step - loss: 19.1945 - val_loss: 24.6519
Epoch 322/10000
4/4 [==============================] - 0s 15ms/step - loss: 24.0397 - val_loss: 25.2703
Epoch 323/10000
4/4 [==============================] - 0s 13ms/step - loss: 20.4423 - val_loss: 31.3263
Epoch 324/10000
4/4 [==============================] - 0s 14ms/step - loss: 20.8699 - val_loss: 29.4044
Epoch 325/10000
4/4 [==============================] - 0s 13ms/step - loss: 25.0967 - val_loss: 32.4205
Epoch 326/10000
4/4 [==============================] - 0s 12ms/s

Epoch 396/10000
4/4 [==============================] - 0s 12ms/step - loss: 21.0178 - val_loss: 25.6438
Epoch 397/10000
4/4 [==============================] - 0s 14ms/step - loss: 19.0504 - val_loss: 24.0038
Epoch 398/10000
4/4 [==============================] - 0s 14ms/step - loss: 18.5232 - val_loss: 28.7556
Epoch 399/10000
4/4 [==============================] - 0s 14ms/step - loss: 16.7405 - val_loss: 23.7970
Epoch 400/10000
4/4 [==============================] - 0s 14ms/step - loss: 16.1828 - val_loss: 21.8223
Epoch 401/10000
4/4 [==============================] - 0s 14ms/step - loss: 17.2551 - val_loss: 24.9961
Epoch 402/10000
4/4 [==============================] - 0s 13ms/step - loss: 14.2124 - val_loss: 34.5463
Epoch 403/10000
4/4 [==============================] - 0s 13ms/step - loss: 18.0473 - val_loss: 23.6639
Epoch 404/10000
4/4 [==============================] - 0s 13ms/step - loss: 17.6869 - val_loss: 26.1433
Epoch 405/10000
4/4 [==============================] - 0s 12ms/s

...

Epoch 2994/10000
4/4 [==============================] - 0s 14ms/step - loss: 3.8738 - val_loss: 15.8522
Epoch 2995/10000
4/4 [==============================] - 0s 12ms/step - loss: 3.9420 - val_loss: 23.2525
Epoch 2996/10000
4/4 [==============================] - 0s 13ms/step - loss: 3.7402 - val_loss: 24.1231
Epoch 2997/10000
4/4 [==============================] - 0s 14ms/step - loss: 3.8134 - val_loss: 17.3437
Epoch 2998/10000
4/4 [==============================] - 0s 14ms/step - loss: 5.2610 - val_loss: 21.2889
Epoch 2999/10000
4/4 [==============================] - 0s 14ms/step - loss: 4.9781 - val_loss: 19.0817
Epoch 3000/10000
4/4 [==============================] - 0s 11ms/step - loss: 4.0106 - val_loss: 17.3983
Epoch 3001/10000
4/4 [==============================] - 0s 12ms/step - loss: 3.8926 - val_loss: 24.4067
Epoch 3002/10000
4/4 [==============================] - 0s 13ms/step - loss: 3.5325 - val_loss: 16.7263
Epoch 3003/10000
4/4 [==============================] - 0s 13ms/

Epoch 3073/10000
4/4 [==============================] - 0s 13ms/step - loss: 4.0397 - val_loss: 17.1249
Epoch 3074/10000
4/4 [==============================] - 0s 13ms/step - loss: 4.5585 - val_loss: 25.1029
Epoch 3075/10000
4/4 [==============================] - 0s 11ms/step - loss: 4.6221 - val_loss: 24.2250
Epoch 3076/10000
4/4 [==============================] - 0s 14ms/step - loss: 5.1134 - val_loss: 22.5252
Epoch 3077/10000
4/4 [==============================] - 0s 13ms/step - loss: 4.1668 - val_loss: 23.2674
Epoch 3078/10000
4/4 [==============================] - 0s 14ms/step - loss: 3.4839 - val_loss: 22.8108
Epoch 3079/10000
4/4 [==============================] - 0s 12ms/step - loss: 3.2104 - val_loss: 21.8877
Epoch 3080/10000
4/4 [==============================] - 0s 13ms/step - loss: 3.0345 - val_loss: 22.8740
Epoch 3081/10000
4/4 [==============================] - 0s 13ms/step - loss: 3.5733 - val_loss: 20.0418
Epoch 3082/10000
4/4 [==============================] - 0s 12ms/

Epoch 3152/10000
4/4 [==============================] - 0s 13ms/step - loss: 3.5525 - val_loss: 17.3273
Epoch 3153/10000
4/4 [==============================] - 0s 13ms/step - loss: 3.0053 - val_loss: 16.8285
Epoch 3154/10000
4/4 [==============================] - 0s 13ms/step - loss: 2.6172 - val_loss: 18.0754
Epoch 3155/10000
4/4 [==============================] - 0s 15ms/step - loss: 3.8506 - val_loss: 16.9006
Epoch 3156/10000
4/4 [==============================] - 0s 14ms/step - loss: 2.3288 - val_loss: 18.2909
Epoch 3157/10000
4/4 [==============================] - 0s 14ms/step - loss: 3.6514 - val_loss: 17.9321
Epoch 3158/10000
4/4 [==============================] - 0s 13ms/step - loss: 3.0403 - val_loss: 18.3902
Epoch 3159/10000
4/4 [==============================] - 0s 13ms/step - loss: 3.3368 - val_loss: 18.3725
Epoch 3160/10000
4/4 [==============================] - 0s 14ms/step - loss: 3.2428 - val_loss: 18.9598
Epoch 3161/10000
4/4 [==============================] - 0s 13ms/

Epoch 3231/10000
4/4 [==============================] - 0s 12ms/step - loss: 3.6294 - val_loss: 17.6341
Epoch 3232/10000
4/4 [==============================] - 0s 13ms/step - loss: 2.8502 - val_loss: 18.2642
Epoch 3233/10000
4/4 [==============================] - 0s 13ms/step - loss: 2.8912 - val_loss: 17.0483
Epoch 3234/10000
4/4 [==============================] - 0s 13ms/step - loss: 3.0152 - val_loss: 18.6557
Epoch 3235/10000
4/4 [==============================] - 0s 15ms/step - loss: 4.1089 - val_loss: 17.5266
Epoch 3236/10000
4/4 [==============================] - 0s 14ms/step - loss: 2.7368 - val_loss: 18.9284
Epoch 3237/10000
4/4 [==============================] - 0s 13ms/step - loss: 3.2482 - val_loss: 17.5963
Epoch 3238/10000
4/4 [==============================] - 0s 13ms/step - loss: 2.7792 - val_loss: 17.2696
Epoch 3239/10000
4/4 [==============================] - 0s 12ms/step - loss: 3.0333 - val_loss: 18.0748
Epoch 3240/10000
4/4 [==============================] - 0s 13ms/

Epoch 3308: early stopping


In [53]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [54]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 17ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,54.907639,54.69239,54.470581,53.594666,52.000385,50.846516,50.194813,49.552071,49.315979,49.181236,48.964439,48.548916,49.045219,48.404549,48.011505,48.525333,48.78019,48.910168,48.044006,47.473587,47.756866,47.616524,47.599495,47.646873,47.773197,48.007069,48.455025,48.903168,49.366173,50.396332
Target,48.75,59.738,60.303,51.009,48.874,48.506,42.721,32.801,37.638,37.358,31.339,45.311,42.085,48.52,41.365,43.813,43.816,38.245,46.081,38.703,36.037,38.43,44.471,37.884,51.585,54.306,52.511,59.472,54.022,49.791
Error,6.157639,5.045609,5.83242,2.585667,3.126385,2.340515,7.473812,16.751072,11.677979,11.823235,17.625439,3.237915,6.96022,0.115452,6.646503,4.712334,4.964188,10.665169,1.963005,8.770588,11.719868,9.186523,3.128494,9.762875,3.811802,6.298931,4.055977,10.568832,4.655827,0.605331


In [55]:
display(mae)
display(mape)

6.7423196

0.16498424

In [56]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [57]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

NameError: name 'test_target' is not defined